In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bert/tensorflow2/en-uncased-l-12-h-768-a-12/3/saved_model.pb
/kaggle/input/bert/tensorflow2/en-uncased-l-12-h-768-a-12/3/assets/vocab.txt
/kaggle/input/bert/tensorflow2/en-uncased-l-12-h-768-a-12/3/variables/variables.index
/kaggle/input/bert/tensorflow2/en-uncased-l-12-h-768-a-12/3/variables/variables.data-00000-of-00001
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/saved_model.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/keras_metadata.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/assets/vocab.txt
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.index
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.data-00000-of-00001
/kaggle/input/datasetspam/spam.csv


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

2024-01-30 20:13:53.131153: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 20:13:53.131267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 20:13:53.289141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df= pd.read_csv("/kaggle/input/datasetspam/spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

**THERE IS SOME IMBLANCE IN THE DATASET**

In [5]:
df_spam=df[df['Category']=='spam']

In [6]:
df_ham=df[df['Category']=='ham']

In [7]:
df_ham_downsampled=df_ham.sample(df_spam.shape[0])

In [8]:
df_balanced=pd.concat([df_spam,df_ham_downsampled])

In [9]:
df_balanced['Category'].value_counts()

Category
spam    747
ham     747
Name: count, dtype: int64

In [10]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x :1 if x=='spam' else 0)

In [11]:
df_balanced.sample(5)

,Category,Message,spam
1438,ham,Wat so late still early mah. Or we juz go 4 di...,0
163,ham,I'm so in love with you. I'm excited each day ...,0
4904,ham,I;m reaching in another 2 stops.,0
3352,ham,At what time are you coming.,0
2316,ham,That's significant but dont worry.,0


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test= train_test_split(df_balanced['Message'],df_balanced['spam'],stratify=df_balanced['spam'])

In [13]:
X_train.head(10)

3217    URGENT! We are trying to contact U. Todays dra...
789     5 Free Top Polyphonic Tones call 087018728737,...
3126    1st wk FREE! Gr8 tones str8 2 u each wk. Txt N...
2374    XMAS Prize draws! We are trying to contact U. ...
4348    U 447801259231 have a secret admirer who is lo...
5540    ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
30      Wait that's still not all that clear, were you...
4249    accordingly. I repeat, just text the word ok o...
1722    Am watching house – very entertaining – am get...
4456    Storming msg: Wen u lift d phne, u say "HELLO"...
Name: Message, dtype: object

In [14]:
bert_preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
bert_encoder = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/3")

Attaching model 'tensorflow/bert/tensorflow2/en-uncased-preprocess/3' to your Kaggle notebook...
Attaching model 'tensorflow/bert/tensorflow2/en-uncased-l-12-h-768-a-12/3' to your Kaggle notebook...


In [15]:
def get_sentence_embeding(sentences):
    preprocessed_text= bert_preprocessor(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [16]:
e= get_sentence_embeding([
    "banana",
    "grapes","mango","jeff bezos","elon musk","bill gates"
])

In [17]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[-0.7606916 , -0.1421939 ,  0.4960467 , ...,  0.42165366,
        -0.5322139 ,  0.80312145],
       [-0.86023206, -0.21242917,  0.49156943, ...,  0.3979806 ,
        -0.6050627 ,  0.8447165 ],
       [-0.7128858 , -0.15463899,  0.38401696, ...,  0.35278735,
        -0.50991327,  0.7347407 ],
       [-0.8253348 , -0.3555059 , -0.5906971 , ..., -0.01613699,
        -0.6141758 ,  0.8723029 ],
       [-0.750413  , -0.26812592, -0.2668958 , ...,  0.02839475,
        -0.59380937,  0.79749817],
       [-0.7854431 , -0.29949626,  0.41027677, ...,  0.5222551 ,
        -0.4957349 ,  0.81507415]], dtype=float32)>

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
cosine_similarity([e[4]],[e[3]])

array([[0.9872034]], dtype=float32)

In [20]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text=bert_preprocessor(text_input)
outputs= bert_encoder(preprocessed_text)

In [21]:
l=tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)


In [22]:
model=tf.keras.Model(inputs=[text_input],outputs=[l])

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_word_ids': (None,    0         ['input_1[0][0]']             
                             128),                                                                
                              'input_type_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                                

In [24]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='Precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=METRICS)

In [25]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10
 1/35 [..............................] - ETA: 5:30 - loss: 0.6295 - accuracy: 0.5938 - Precision: 0.6333 - recall: 0.9048

I0000 00:00:1706645680.554736      72 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35/35 [==============================] - 20s 294ms/step - loss: 0.6240 - accuracy: 0.6804 - Precision: 0.6836 - recall: 0.6714
Epoch 2/10
35/35 [==============================] - 10s 297ms/step - loss: 0.4891 - accuracy: 0.8402 - Precision: 0.8313 - recall: 0.8536
Epoch 3/10
35/35 [==============================] - 11s 300ms/step - loss: 0.4234 - accuracy: 0.8687 - Precision: 0.8655 - recall: 0.8732
Epoch 4/10
35/35 [==============================] - 11s 304ms/step - loss: 0.3763 - accuracy: 0.8821 - Precision: 0.8808 - recall: 0.8839
Epoch 5/10
35/35 [==============================] - 11s 308ms/step - loss: 0.3435 - accuracy: 0.8964 - Precision: 0.8922 - recall: 0.9018
Epoch 6/10
35/35 [==============================] - 11s 312ms/step - loss: 0.3250 - accuracy: 0.9045 - Precision: 0.8995 - recall: 0.9107
Epoch 7/10
35/35 [==============================] - 11s 319ms/step - loss: 0.3085 - accuracy: 0.9027 - Precision: 0.9005 - recall: 0.9054
Epoch 8/10
35/35 [===========================

In [26]:
 y_predicted=model.predict(X_test)

12/12 [==============================] - 4s 310ms/step


In [27]:
y_predicted=y_predicted.flatten()

In [28]:
y_predicted

array([0.09060279, 0.91048074, 0.61975735, 0.22444066, 0.8199088 ,
       0.65779805, 0.04106794, 0.56966895, 0.07867905, 0.1906043 ,
       0.01329728, 0.788162  , 0.6990626 , 0.00599122, 0.9191588 ,
       0.97706133, 0.8474964 , 0.05352975, 0.5269097 , 0.9548245 ,
       0.18105742, 0.36369088, 0.41217127, 0.06230614, 0.90033495,
       0.6865837 , 0.918583  , 0.0797281 , 0.03727516, 0.96135414,
       0.32672784, 0.04080017, 0.23086147, 0.04074951, 0.05306548,
       0.45613763, 0.26944074, 0.9446239 , 0.8216216 , 0.7149376 ,
       0.7750759 , 0.894094  , 0.9671778 , 0.9049389 , 0.6022083 ,
       0.40823227, 0.8716264 , 0.0241708 , 0.6325326 , 0.95289063,
       0.51216704, 0.1748586 , 0.46453324, 0.29386318, 0.02907345,
       0.09099828, 0.35457957, 0.01350845, 0.0435731 , 0.04241963,
       0.9421133 , 0.67037606, 0.04648522, 0.7994902 , 0.9340436 ,
       0.7226654 , 0.04070574, 0.60257006, 0.84108025, 0.06311991,
       0.10708261, 0.0330059 , 0.09435981, 0.40065932, 0.00967

In [29]:
import numpy as np
y_predicted=np.where(y_predicted>0.5,1,0)
y_predicted

array([0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,

In [30]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
arr=model.predict(reviews)

1/1 [==============================] - 1s 838ms/step


In [31]:
for x in arr:
    if x<0.5:
        print("NOT-SPAM")
    else:
        print("SPAM")

SPAM
SPAM
SPAM
NOT-SPAM
NOT-SPAM
